# Introducción a la programación de Computación Cuántica con ProjectQ

## Índice



1.   Descripción, referencias y documentación. Instalación
2.   Primer ejemplo: Engines y backends, Sintaxis básica
3.   Quantum registers y Gates (Puertas) (Gates comunes + Gates especiales (All, C) + Gates generadoras (BasicGate, MatrixGate) + Qubit Operator + Gates de funciones especiales (QPE, QAA))
4.   Declaraciones Especiales (Control, Loop, Compute, Uncompute, Dagger)
5.   El simulador: Extraer amplitudes y probabilidades, valores esperados



## Descripción, referencias y documentación. Instalación

ProjectQ es un framework python open source para quantum computing creado inicialmente en el ETH de Zurich por Thomas Häner y Damian S. Steiger en el grupo del Prof. Dr. Matthias Troyer. Su intención es proveer las herramientas para facilitar el desarollo, implementación, pruebas, depuración y ejecución de algoritmos cuánticos usando tanto hardware clásico como dispositivos cuánticos.

* Documentación: https://projectq.readthedocs.io/en/latest/
* GitHub (incluye manuales e instrucciones para distintas cosas): https://github.com/ProjectQ-Framework/ProjectQ
* Versión actual: 0.5.1 liberada el 5 de junio 20202 https://github.com/ProjectQ-Framework/ProjectQ/releases



In [ ]:
!pip install projectq

## Primer ejemplo: Engines y backends, Sintaxis básica

### Backend por defecto: el simulador

In [ ]:
from projectq import MainEngine  # import the main compiler engine
from projectq.ops import H, Measure  # import the operations we want to perform (Hadamard and measurement)

eng1 = MainEngine()  # create a default compiler (the back-end is a simulator)
qubit = eng1.allocate_qubit()  # allocate 1 qubit

H | qubit  # apply a Hadamard gate
Measure | qubit  # measure the qubit

eng1.flush()  # flush all gates (and execute measurements)
print("Measured {}".format(int(qubit)))  # output measurement result

del eng1

Measured 0


### Otros backends: IBM Quantum Experience, AQT Trapped Ion, CircuitDrawer, ResourceCounter

In [ ]:
# ResourceCounter
from projectq.backends import ResourceCounter
from projectq.ops import QFT, CNOT, Swap
from projectq.setups import linear # Solo lo nombro

compiler_engines = linear.get_engine_list(num_qubits=16,
                                          one_qubit_gates='any',
                                          two_qubit_gates=(CNOT, Swap))

resource_counter = ResourceCounter()
engRC = MainEngine(backend=resource_counter, engine_list=compiler_engines)

qureg = engRC.allocate_qureg(16)
QFT | qureg
engRC.flush()

print(resource_counter)

del engRC


In [ ]:
# CircuitDrawerMatplotlib
from projectq.backends import CircuitDrawerMatplotlib # Existe CircuitDrawer que crea latex
from projectq.cengines import DummyEngine
from projectq.ops import (H, X, Rx, CNOT, Swap, Measure, Command, BasicGate)

%matplotlib inline
import matplotlib.pyplot as plt

# Esto lo miramos luego
class MyGate(BasicGate):
    def __init__(self, *args):
        BasicGate.__init__(self)
        self.params = args

    def __str__(self):
        param_str = '{}'.format(self.params[0])
        for param in self.params[1:]:
            param_str += ',{}'.format(param)
        return str(self.__class__.__name__) + "(" + param_str + ")"

backend = DummyEngine()

drawer = CircuitDrawerMatplotlib()

engCDM = MainEngine(backend, [drawer]) # Uso de Next Engine
qureg = engCDM.allocate_qureg(3)
H | qureg[1]
H | qureg[0]
X | qureg[0]
Rx(1) | qureg[1]
CNOT | (qureg[0], qureg[1])
Swap | (qureg[0], qureg[1])
MyGate(1.2) | qureg[2]
MyGate(1.23456789) | qureg[2]
MyGate(1.23456789, 2.3456789) | qureg[2]
MyGate(1.23456789, 'aaaaaaaa', 'bbb', 2.34) | qureg[2]
X | qureg[0]

qubit_lines = drawer.draw()

del engCDM


In [ ]:
# IBM Quantum Experience (este no lo ejecuto, hace falta un TOKEN)
import projectq.setups.ibm
from projectq.backends import IBMBackend

token='MY_TOKEN'
device='ibmq_16_melbourne'
compiler_engines = projectq.setups.ibm.get_engine_list(token=token,device=device)
eng = MainEngine(IBMBackend(token=token, use_hardware=True, num_runs=1024,
                            verbose=False, device=device),
                            engine_list=compiler_engines)

## Quantum registers y Gates (Puertas)

### Quantum registers

In [ ]:
# Ya los hemos visto antes
'''
engCDM = MainEngine(backend, [drawer]) # Uso de Next Engine
qureg = engCDM.allocate_qureg(3)
H | qureg[1]
H | qureg[0]
X | qureg[0]
Rx(1) | qureg[1]
CNOT | (qureg[0], qureg[1])
'''

### Gates (puertas) comunes
Pordeis encontrar todas las puertas en https://projectq.readthedocs.io/en/latest/projectq.ops.html#

Entre ellas las operaciones más comunes y que habeis visto, y que hemos visto como se usan en celdas anteriores:
Rotaciones, Paulis, Hadamard, en general Clifford, incluyendo la operación de Measure.
Pero tambien non Clifford: T, S
También 2qubit gates: CNOT, Swap, controled-rotations, y uan general control(gate).
La nomenclatura general es

    Gate | qubit
    Gate | [qubit0, qubit1]
    Gate | qureg
    Gate | (qubit, )
    Gate | (qureg, qubit)



In [ ]:
# Ejemplos (ejercicio: hacer que estas puertas funcionen y Medir al final)
'''
H | qureg[1]
H | qureg[0]
X | qureg[0]
Rx(1.23456789) | qureg[1] # El número es el angulo a rotar
CNOT | (qureg[0], qureg[1])
C(NOT) | (qureg[0], qureg[1])
Toffoli = C(NOT,2) = C(CNOT) # el numero es la cantidad de controls, en general C(Gate,n), con n=1 por defecto
Measure | qubit

'''

####Los ángulos
Veamos como están definidas esas puertas para entender como hemos de tratar los

In [ ]:
'''
class Rz(BasicRotationGate):
    """ RotationZ gate class """
    @property
    def matrix(self):
        return np.matrix([[cmath.exp(-.5 * 1j * self.angle), 0],
                          [0, cmath.exp(.5 * 1j * self.angle)]])
'''

###Puertas especiales (All, C)
*  C (Control) ya la hemos visto, C(Gate), C(X), C(Z), C(H) (C(X) = CNOT) hay sinónimos de operaciones
*  All(Gate) nos permite ejecutar la misma puerta sobre todos los registros de qubit que pongamos

In [ ]:
# Haced alguna prueba
from projectq.ops import C, All

engGates = MainEngine()
quregGates1 = engGates.allocate_qureg(2)
quregGates2 = engGates.allocate_qureg(4)

X | quregGates1[1]
C(H) | [quregGates1[1], quregGates1[0]]
#C(H) | (quregGates1[1], quregGates1[0])
All(Measure) | quregGates1

engGates.flush()
print("Measured {}".format([int(q) for q in quregGates1])) # Al estar medidos ya estan colapsados

H | quregGates2[0]
All(H) | quregGates2[1:]
All(Measure) | quregGates2

engGates.flush()
print("Measured {}".format([int(q) for q in quregGates2]))

del engGates



###Puertas Generadoras (BasicGate, MatrixGate)
Nos permiten generar heredando cualquier puerta (Unitaria) que nos imaginemos por ejemplo para combinar varias puertas en una función

#### Basic Gates

In [ ]:
# BasicGate (no recomendado!!!!!)
# Hemos visto esta antes... pero no hace nada
class MyGate(BasicGate):
    def __init__(self, *args):
        BasicGate.__init__(self)
        self.params = args

    def __str__(self):
        param_str = '{}'.format(self.params[0])
        for param in self.params[1:]:
            param_str += ',{}'.format(param)
        return str(self.__class__.__name__) + "(" + param_str + ")"

In [ ]:
# Ejemplo de como se usa para generar una de las puertas en ops (de nuevo, no recomendado!!!)
class SqrtSwapGate(BasicGate):
    """ Square-root Swap gate class """
    def __init__(self):
        BasicGate.__init__(self)
        self.interchangeable_qubit_indices = [[0, 1]]

    def __str__(self):
        return "SqrtSwap"

    @property
    def matrix(self):
        return np.matrix([[1, 0, 0, 0],
                          [0, 0.5+0.5j, 0.5-0.5j, 0],
                          [0, 0.5-0.5j, 0.5+0.5j, 0],
                          [0, 0, 0, 1]])

Hay más Basic Gates: BasicRotatinGate, BasicPhaseGate... No es recomendado usar ninguna de ellas para generar nuestras puertas...

#### MatrixGate
Define una puerta (clase como hemos visto) mediante una matriz. 

In [ ]:
from projectq.ops import MatrixGate
unitary_op = MatrixGate([[0, 1, 0, 0],
                         [1, 0, 0, 0],
                         [0, 0, 0, 1],
                         [0, 0, 1, 0]])

dir(unitary_op) # La puerta definida hereda los métodos de MatrixGate

In [ ]:
engM = MainEngine()
quregM1 = engM.allocate_qureg(2)

H | quregM1[0]
CNOT | (quregM1[0], quregM1[1]) # esto hace un Bell pair

unitary_op | quregM1

All(Measure) | quregM1

engM.flush()
print("Measured {}".format([int(q) for q in quregM1]))

quregM2 = engM.allocate_qureg(2)

H | quregM2[0]
CNOT | (quregM2[0], quregM2[1]) # esto hace un Bell pair

unitary_op | quregM2
unitary_op.get_inverse() | quregM2  # No hemos definido la inversa en ningun lado, nos lo da gratis

All(Measure) | quregM2

engM.flush()
print("Measured {}".format([int(q) for q in quregM2]))

del engM



###Qubit Operator
Es una forma de construir Pauli operators que además nos sirve para crear Hamiltonianos y su evolución temporal. Veamoslo con los ejemplos de la documentación


In [ ]:
# QubitOperator

from projectq.ops import QubitOperator

engO = MainEngine()
quregO1 = engO.allocate_qureg(6)
QubitOperator('X0 X5', 1.j) | quregO1  # Applies X to qubit 0 and 5
                                     # with an additional global phase
                                     # of 1.j
All(Measure) | quregO1

engO.flush()
print("Measured {}".format([int(q) for q in quregO1]))

# Tambien se puede definir

ham = ((QubitOperator('X0 Y3', 0.5) + 0.6 * QubitOperator('X0 Y3')))

# Equivalently
ham2 = QubitOperator('X0 Y3', 0.5)
ham2 += 0.6 * QubitOperator('X0 Y3')

# Pero lo que no se puede hacer es

quregO2 = engO.allocate_qureg(6)

ham | quregO2

All(Measure) | quregO2

engO.flush()
print("Measured {}".format([int(q) for q in quregO2]))

###TimeEvolution

Puerta para la evolución temporal bajo un Hamiltoniano (objeto del tipo QubitOperator)

La puerta TimeEvolution es la evolución temporal unitaria del propagador exp(-i * H * t), donde H es el Hamiltoniano del sistema y t es el tiempo.

In [ ]:
# TimeEvolution

from projectq.ops import TimeEvolution

quregO3 = engO.allocate_qureg(5)
X | quregO3[1]
hamiltonian = 0.5 * QubitOperator("X0 Z1 Y4")

TimeEvolution(time=2.5, hamiltonian=hamiltonian) | quregO3 # Probar con distintos tiempos (0.5, 2.5)

All(Measure) | quregO3

engO.flush()
print("Measured {}".format([int(q) for q in quregO3]))


In [ ]:
del engO

###Puertas de funciones especiales (QFT, QPE, QAA)
Son realmente algoritmos (Quatum Fourier Transform, Quantum Phase Estimation, Quantum Amplitude Amplification). ProjectQ los trata como puertas que aplican a quantum registers. Para ver como se usan es necesario revisar la documentación. Veamos alguno.

In [ ]:
# QFT

from projectq.ops import QFT

engQ = MainEngine()
quregQFT = engQ.allocate_qureg(5)

All(H) | quregQFT

QFT | quregQFT # Por supuesto existe la inversa

All(Measure) | quregQFT

engQ.flush()
print("Measured {}".format([int(q) for q in quregQFT]))


In [ ]:
# QPE, sumper sencillo copiando el ejemplo en la documentación

from projectq.ops import QPE, Ph
import cmath

n_qpe_ancillas = 3
qpe_ancillas = engQ.allocate_qureg(n_qpe_ancillas)
system_qubits = engQ.allocate_qureg(1)
angle = cmath.pi*2.*0.125
U = Ph(angle) # unitary_specfic_to_the_problem()

# Apply Quantum Phase Estimation
QPE(U) | (qpe_ancillas, system_qubits)

All(Measure) | qpe_ancillas
# Compute the phase from the ancilla measurement
#(https://en.wikipedia.org/wiki/Quantum_phase_estimation_algorithm)
phasebinlist = [int(q) for q in qpe_ancillas]
phase_in_bin = ''.join(str(j) for j in phasebinlist)
phase_int = int(phase_in_bin,2)
phase = phase_int / (2 ** n_qpe_ancillas)
print (phase)


In [ ]:
del engQ

##Declaraciones Especiales (Control, Loop, Compute, Uncompute, Dagger)
Declaraciones que nos ayudan a hacer un código nás eficiente

In [ ]:
# Control
from projectq.meta import Control, Loop, Compute, Uncompute, Dagger

engD = MainEngine()

quregC = engD.allocate_qureg(4)

All(X) | quregC[0:1]

with Control(engD, qureg[:1]):
  X | quregC [2]
  All(H) | qureg[2:3]

All(Measure) | quregC
engD.flush()
del quregC # Deallocate de los qubits


In [ ]:
# Loop

with Loop(engD,3):
  quregL = engD.allocate_qureg(2)
  All(H) | quregL
  All(Measure) | quregL
  engD.flush()
  #print("Measured {}".format([int(q) for q in quregL]))
  del quregL # Necesario hacer el deallocate de los qubits si se "allocan" en el loop 
  
# Por supuesto se puede usar un with Control dentro de un Loop

In [ ]:
# Dagger
# Invierte un grupo de código
# También aplica el deallocate dentro del whith si se crean qubits dentro

quregD = engD.allocate_qureg(3)

X | quregD[0]
All(H) | quregD[::1]
CNOT | (qureg[2], qureg[1])

with Dagger(engD):
  X | quregD[0]
  All(H) | quregD[::1]
  CNOT | (qureg[2], qureg[1])

All(Measure) | quregD
engD.flush()
print("Measured {}".format([int(q) for q in quregD]))

del quregD


In [ ]:
# Compute y Uncompute
# Funcionan conjuntamente habitualmente. Comienza un grupo de computación y lo deshace

quregCU = engD.allocate_qureg(4)
controlCU = engD.allocate_qubit()

X | controlCU
H | controlCU

X | quregCU[0]

with Compute(engD):
  with Control(engD, quregCU[0]):
    All(X) | quregCU[1:]
  H | quregCU[0]
  All(X) | quregCU

with Control(engD, quregCU):
  X | controlCU

Uncompute(engD)

All(Measure) | quregCU
engD.flush()
print("Measured {}".format([int(q) for q in quregCU]))

H | controlCU
Measure | controlCU

engD.flush()
print("Measured {}".format([int(q) for q in controlCU]))

del quregCU
del controlCU

In [ ]:
del engD

## El simulador: Extraer amplitudes y probabilidades, valores esperados
El simulador mantiene toda la información de los estados, por tanto podemos acceder a ella (cosa que no podemos hacer en un dispositivo real)

In [ ]:
# Obtener amplitudes
from projectq.ops import Ry, QubitOperator
from projectq.meta import Control
import math

engSim = MainEngine()
quregSim = engSim.allocate_qureg(4)

def complex_algorithm(eng, qreg):
    All(H) | qreg
    with Control(eng, qreg[0]):
        All(X) | qreg[1:]
    All(Ry(math.pi / 4)) | qreg[1:]
    with Control(eng, qreg[-1]):
        All(X) | qreg[1:-1]

complex_algorithm(engSim, quregSim)

engSim.flush()
amplitude1 = engSim.backend.get_amplitude('0101', quregSim)
amplitude2 = engSim.backend.get_amplitude('1010', quregSim)

print('0101: ', amplitude1,'\n1010: ', amplitude2)


In [ ]:
# Obtener probabilidades
prob1 = engSim.backend.get_probability('0101', quregSim)
prob2 = engSim.backend.get_probability('1010', quregSim)

print('0101: ', prob1,'\n1010: ', prob2)

In [ ]:
# Expectation Value
# Obtiene el valor esperado de un operador con respecto al estadoactual de la función de onda

hamiltoniano = QubitOperator('X0 Z1 Z2', 0.2)
expectedvalue = engSim.backend.get_expectation_value(hamiltoniano, quregSim)
print('Valor esperado: ', expectedvalue)

In [ ]:
# Histograma (función añadida en la última versión)
from projectq.libs.hist import histogram

import matplotlib
import matplotlib.pyplot as plt

histogram(engSim.backend, quregSim)
plt.show()

In [ ]:
All(Measure) | quregSim
engSim.flush()

del quregSim